In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/raintomorrow/train_df (2).csv


# Imports

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import operator
warnings.filterwarnings("ignore")
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import resample

# Загрузка и предварительная обработка данных

In [46]:
df = pd.read_csv('/kaggle/input/raintomorrow/train_df (2).csv')
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2014-05-24,Penrith,10.6,24.9,0.0,NaN,NaN,NW,20.0,SSW,...,68.0,47.0,NaN,NaN,NaN,NaN,16.7,23.3,No,No
1,2016-09-04,Dartmoor,6.0,15.1,1.6,NaN,NaN,NW,41.0,NNW,...,94.0,77.0,1023.0,1024.0,NaN,NaN,10.6,13.9,Yes,No
2,2011-01-26,Moree,24.6,41.4,0.0,12.4,12.8,NE,31.0,NNE,...,51.0,24.0,1012.7,1009.6,0.0,1.0,30.1,39.3,No,No
3,2013-08-22,Bendigo,3.7,12.6,6.6,1.4,NaN,NW,41.0,WNW,...,95.0,71.0,1011.3,1009.4,8.0,8.0,8.5,12.2,Yes,Yes
4,2011-10-19,Watsonia,7.6,27.9,0.0,8.0,12.5,N,54.0,NE,...,57.0,36.0,1026.6,1023.1,1.0,1.0,19.8,27.3,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99530,2016-01-12,BadgerysCreek,19.7,34.6,5.0,NaN,NaN,SE,33.0,N,...,65.0,53.0,1010.3,1011.1,NaN,NaN,27.0,29.2,Yes,No
99531,2012-11-23,Mildura,12.4,32.3,0.0,10.2,10.5,NE,24.0,E,...,43.0,19.0,1017.9,1014.4,4.0,6.0,21.2,29.9,No,No
99532,2014-02-19,Cobar,23.0,34.1,0.0,8.2,NaN,W,56.0,NNW,...,73.0,19.0,1002.1,999.2,7.0,1.0,25.6,33.3,No,No
99533,2015-11-09,PerthAirport,14.7,30.4,0.0,7.0,12.4,E,52.0,S,...,58.0,40.0,1019.7,1016.0,1.0,2.0,22.6,28.1,No,No


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99535 entries, 0 to 99534
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           99535 non-null  object 
 1   Location       99535 non-null  object 
 2   MinTemp        99091 non-null  float64
 3   MaxTemp        99317 non-null  float64
 4   Rainfall       98569 non-null  float64
 5   Evaporation    56916 non-null  float64
 6   Sunshine       51928 non-null  float64
 7   WindGustDir    93019 non-null  object 
 8   WindGustSpeed  93059 non-null  float64
 9   WindDir9am     92497 non-null  object 
 10  WindDir3pm     96880 non-null  object 
 11  WindSpeed9am   98603 non-null  float64
 12  WindSpeed3pm   97702 non-null  float64
 13  Humidity9am    98284 non-null  float64
 14  Humidity3pm    96999 non-null  float64
 15  Pressure9am    89721 non-null  float64
 16  Pressure3pm    89726 non-null  float64
 17  Cloud9am       61971 non-null  float64
 18  Cloud3

In [48]:
df.isnull().sum()

Date                 0
Location             0
MinTemp            444
MaxTemp            218
Rainfall           966
Evaporation      42619
Sunshine         47607
WindGustDir       6516
WindGustSpeed     6476
WindDir9am        7038
WindDir3pm        2655
WindSpeed9am       932
WindSpeed3pm      1833
Humidity9am       1251
Humidity3pm       2536
Pressure9am       9814
Pressure3pm       9809
Cloud9am         37564
Cloud3pm         40006
Temp9am            635
Temp3pm           1910
RainToday          966
RainTomorrow         0
dtype: int64

In [49]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.sort_values(by=['Date'], inplace=True)

In [50]:
columns_to_ffill = [
    'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
    'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
    'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
    'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
    'Temp9am', 'Temp3pm'
]

df[columns_to_ffill] = df[columns_to_ffill].ffill()

In [51]:
df = pd.get_dummies(df, columns=['WindGustDir', 
                                 'WindDir9am', 'WindDir3pm'])

In [52]:
df['RainToday'] = df['RainToday'].map({'No': 0, 'Yes': 1})
df['RainTomorrow'] = df['RainTomorrow'].map({'No': 0, 'Yes': 1})

In [53]:
location_coords = {
    "Penrith": (-33.7511, 150.6923),
    "Dartmoor": (-35.2000, 150.1200),
    "Moree": (-29.4972, 149.8333),
    "Bendigo": (-36.7580, 144.2794),
    "Watsonia": (-37.6800, 145.1000),
    "NorahHead": (-33.4167, 151.4333),
    "Hobart": (-42.8821, 147.3272),
    "Albany": (-35.0262, 117.8812),
    "BadgerysCreek": (-33.6729, 150.8333),
    "Ballarat": (-37.5622, 143.8503),
    "SydneyAirport": (-33.9461, 151.1770),
    "Launceston": (-41.4545, 147.1386),
    "Portland": (-38.3614, 141.4717),
    "Wollongong": (-34.4278, 150.8931),
    "Newcastle": (-32.9283, 151.7817),
    "NorfolkIsland": (-29.0408, 167.9594),
    "Brisbane": (-27.4698, 153.0251),
    "MelbourneAirport": (-37.6733, 144.8433),
    "GoldCoast": (-28.0167, 153.4000),
    "MountGinini": (-35.5000, 149.0000),
    "AliceSprings": (-23.6980, 133.8807),
    "Woomera": (-31.0685, 136.8043),
    "PearceRAAF": (-31.8900, 115.8100),
    "Katherine": (-14.4477, 132.2669),
    "Adelaide": (-34.9285, 138.6007),
    "Darwin": (-12.4634, 130.8456),
    "Melbourne": (-37.8136, 144.9631),
    "Williamtown": (-32.7800, 151.7800),
    "Mildura": (-34.2083, 142.1572),
    "Sale": (-38.1038, 147.0710),
    "Albury": (-36.0800, 146.9240),
    "Cairns": (-16.9186, 145.7781),
    "WaggaWagga": (-35.1175, 147.3670),
    "Witchcliffe": (-31.9800, 115.8200),
    "Tuggeranong": (-35.3800, 149.0500),
    "CoffsHarbour": (-30.2960, 153.1160),
    "PerthAirport": (-31.9400, 115.9670),
    "SalmonGums": (-33.9833, 121.0667),
    "Sydney": (-33.8688, 151.2093),
    "MountGambier": (-37.8270, 140.7850),
    "Cobar": (-32.6046, 145.7689),
    "Canberra": (-35.2809, 149.1300),
    "Nuriootpa": (-34.5867, 138.9578),
    "Townsville": (-19.2589, 146.8169),
    "Perth": (-31.9505, 115.8605),
    "Richmond": (-37.7833, 144.9833),
    "Walpole": (-33.6600, 116.4900),
    "Nhil": (-36.0600, 142.4000),
    "Uluru": (-25.3450, 131.0369)
}


In [54]:
df['Coordinates'] = df['Location'].map(location_coords)
df[['Latitude', 'Longitude']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)

In [55]:
df.drop(columns=['Coordinates'], inplace=True)

In [56]:
def fill_missing_rain_today(df): 
    
    df = df.copy()
    
    df_train = df[df['RainToday'].notnull()]
    df_missing = df[df['RainToday'].isnull()]
    
    X = df_train.drop(columns=['RainToday', 'Date', 'RainTomorrow', 'Location'])
    y = df_train['RainToday']  
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    class_report = classification_report(y_val, y_pred)
    
    
    print("Classification Report:")
    print(class_report)
    
    if not df_missing.empty:
        X_missing = df_missing.drop(columns=['RainToday', 'Date', 'RainTomorrow', 'Location'])
        predicted = model.predict(X_missing)
        df.loc[df['RainToday'].isnull(), 'RainToday'] = predicted
    
    df.sort_values('Date', inplace=True)
    
    return df

In [57]:
df = fill_missing_rain_today(df)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     15374
         1.0       1.00      1.00      1.00      4340

    accuracy                           1.00     19714
   macro avg       1.00      1.00      1.00     19714
weighted avg       1.00      1.00      1.00     19714



# Генерация признаков

1. Был ли в предыдущем наблюдении дождь для данной локации

In [58]:
df.sort_values(['Latitude', 'Longitude'], inplace=True)
df['Prev_RainToday'] = df.groupby(['Latitude', 'Longitude'])['RainToday'].shift(1)
df['Prev_RainToday'] = df['Prev_RainToday'].fillna(0)
df['Prev_RainToday'] = df['Prev_RainToday'].astype(bool)

2. Время года

In [59]:
def assign_australian_season(month):
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    elif month in [6, 7, 8]:
        return 'Winter'
    elif month in [9, 10, 11]:
        return 'Spring'

In [60]:
df['Month'] = df['Date'].dt.month
df['Season'] = df['Month'].apply(assign_australian_season)
df.drop(columns=['Month'], inplace=True)

3. Климатический пояс

In [61]:
def assign_aus_climate(lat, lon):
    if lat > -23:
        return "Tropical"
    elif -30 < lat <= -23:
        if 130 <= lon <= 140:
            return "Arid"
        else:
            return "Subtropical"
    elif -35 < lat <= -30:
        return "Mediterranean"
    else:
        return "Temperate"

In [62]:
df['ClimateZone'] = df.apply(lambda row: assign_aus_climate(row['Latitude'], row['Longitude']), axis=1)

4. Близость к окену

In [63]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Радиус Земли в километрах
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [64]:
coastal_points = [
    (-33.8688, 151.2093),  # Sydney
    (-37.8136, 144.9631),  # Melbourne
    (-27.4698, 153.0251),  # Brisbane
    (-31.9505, 115.8605),  # Perth
    (-12.4634, 130.8456),  # Darwin (прибрежный)
    (-34.9285, 138.6007)   # Adelaide (близко к побережью)
]

In [65]:
def is_near_ocean(lat, lon, coastal_points, threshold=100):
    distances = [haversine(lat, lon, coast_lat, coast_lon) for coast_lat, coast_lon in coastal_points]
    return min(distances) < threshold

In [66]:
df['NearOcean'] = df.apply(lambda row: is_near_ocean(row['Latitude'], row['Longitude'], coastal_points, threshold=100), axis=1)

5. Расстояние между городами

In [67]:
def all_location_distances(location):
    lat1, lon1 = location_coords[location]
    
    distances = {
        other_locations: haversine(lat1, lon1, lat2, lon2)
        for other_locations, (lat2, lon2) in location_coords.items() 
        if other_locations != location
    }
    return distances

In [68]:
df["AllLocationsDistances"] = df["Location"].apply(all_location_distances)
distances_df = df["AllLocationsDistances"].apply(pd.Series).add_prefix("Distance_")

In [69]:
df = pd.concat([df.drop("AllLocationsDistances", axis=1), distances_df], axis=1)
df = df.fillna(0)

6. Рельеф

In [70]:
location_relief = {
    "Penrith": "Hilly",
    "Dartmoor": "Hilly",
    "Moree": "Flat",
    "Bendigo": "Hilly",
    "Watsonia": "Flat",
    "NorahHead": "Flat",
    "Hobart": "Hilly",
    "Albany": "Hilly",
    "BadgerysCreek": "Flat",
    "Ballarat": "Hilly",
    "SydneyAirport": "Flat",
    "Launceston": "Hilly",
    "Portland": "Flat",
    "Wollongong": "Hilly",
    "Newcastle": "Flat",
    "NorfolkIsland": "Flat",
    "Brisbane": "Flat",
    "MelbourneAirport": "Flat",
    "GoldCoast": "Flat",
    "MountGinini": "Mountainous",
    "AliceSprings": "Flat",
    "Woomera": "Flat",
    "PearceRAAF": "Flat",
    "Katherine": "Flat",
    "Adelaide": "Flat",
    "Darwin": "Flat",
    "Melbourne": "Flat",
    "Williamtown": "Flat",
    "Mildura": "Flat",
    "Sale": "Flat",
    "Albury": "Hilly",
    "Cairns": "Flat",
    "WaggaWagga": "Flat",
    "Witchcliffe": "Flat",
    "Tuggeranong": "Hilly",
    "CoffsHarbour": "Flat",
    "PerthAirport": "Flat",
    "SalmonGums": "Flat",
    "Sydney": "Flat",
    "MountGambier": "Hilly",
    "Cobar": "Flat",
    "Canberra": "Hilly",
    "Nuriootpa": "Flat",
    "Townsville": "Flat",
    "Perth": "Flat",
    "Richmond": "Flat",
    "Walpole": "Hilly",
    "Nhil": "Flat",
    "Uluru": "Flat"
}

In [71]:
df['Relief'] = df['Location'].map(location_relief)

In [72]:
df = pd.get_dummies(df, columns=['Season', 'ClimateZone', 'Relief'])

# Разбиение данных и обучение модели

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99535 entries, 70799 to 51511
Columns: 133 entries, Date to Relief_Mountainous
dtypes: bool(62), datetime64[ns](1), float64(68), int64(1), object(1)
memory usage: 60.6+ MB


In [74]:
df = df.astype({col: 'int' for col in df.select_dtypes(include=['bool']).columns})
df = df.drop(columns=df.select_dtypes(include=['datetime64', 'object']).columns)

In [75]:
minority_class = df[df["RainTomorrow"] == 1]
balanced_minority = resample(minority_class, replace=True, n_samples=df["RainTomorrow"].value_counts()[0], random_state=42)
df = pd.concat([df[df["RainTomorrow"] == 0], balanced_minority])

In [76]:
X=df.drop('RainTomorrow', axis=1)
y=df["RainTomorrow"]

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [79]:
batch_size = 16
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=True)

In [80]:
class FlexibleMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(FlexibleMLP, self).__init__()

        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 32)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(32, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)  
        return x

In [81]:
input_size=X_train.shape[1]
output_size = len(set(y))

In [82]:
model = FlexibleMLP(input_size, output_size)

In [83]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [84]:
num_epochs=100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss {loss.item():.4f}")

Epoch 1, Loss 1.1806
Epoch 2, Loss 0.3392
Epoch 3, Loss 0.0816
Epoch 4, Loss 0.0671
Epoch 5, Loss 0.1415
Epoch 6, Loss 0.3007
Epoch 7, Loss 0.1352
Epoch 8, Loss 0.1795
Epoch 9, Loss 0.0822
Epoch 10, Loss 0.1482
Epoch 11, Loss 0.2921
Epoch 12, Loss 0.1126
Epoch 13, Loss 0.0417
Epoch 14, Loss 0.3323
Epoch 15, Loss 0.4540
Epoch 16, Loss 0.6212
Epoch 17, Loss 0.1029
Epoch 18, Loss 2.1910
Epoch 19, Loss 0.4392
Epoch 20, Loss 0.5473
Epoch 21, Loss 1.2560
Epoch 22, Loss 0.3137
Epoch 23, Loss 0.8760
Epoch 24, Loss 0.5774
Epoch 25, Loss 0.5650
Epoch 26, Loss 0.7376
Epoch 27, Loss 0.3243
Epoch 28, Loss 0.4867
Epoch 29, Loss 0.8293
Epoch 30, Loss 0.0786
Epoch 31, Loss 0.3046
Epoch 32, Loss 0.0923
Epoch 33, Loss 0.1861
Epoch 34, Loss 0.2829
Epoch 35, Loss 0.1579
Epoch 36, Loss 0.1432
Epoch 37, Loss 0.1398
Epoch 38, Loss 1.3920
Epoch 39, Loss 0.7746
Epoch 40, Loss 0.1555
Epoch 41, Loss 1.0752
Epoch 42, Loss 0.4320
Epoch 43, Loss 2.1440
Epoch 44, Loss 0.5184
Epoch 45, Loss 0.0316
Epoch 46, Loss 1.01

In [85]:
model.eval()
y_pred=[]
y_true=[]
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        predictions = torch.argmax(outputs, dim=1)
        y_pred.extend(predictions.numpy())
        y_true.extend(batch_y.numpy())

In [86]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.84      0.78     15317
           1       0.81      0.70      0.75     15572

    accuracy                           0.77     30889
   macro avg       0.77      0.77      0.76     30889
weighted avg       0.77      0.77      0.76     30889

